In [4]:
from sqlalchemy import create_engine
import pandas as pd
import openpyxl

In [22]:
engine = create_engine(r'mssql+pyodbc://LAPTOP-NKOE7QN2/SDPL?driver=ODBC+Driver+17+for+SQL+Server&Trusted_Connection=yes')

df = pd.read_sql("""select cast(INVOICEDATE as date) INVOICEDATE, CONTACTPERSONID, FULL_NAME, ACX_MOBILE, ACX_PHONE, INVENTSITEID 
                    from vw_bi_itemsales
                    where cast(INVOICEDATE as date) between '2024-07-01' and '2024-12-31'
                    and INVENTSITEID in ('KLP', 'SL2', 'GAR', 'SNT', 'SOD', 'BIR', 'CHP', 'SLK', 'MRD', 'BEH',
                                         'ASN', 'BAN', 'BKP', 'BAS', 'BHT', 'COB', 'DNK', 'DIH', 'DGR', 'GHY',
                                         'HWH', 'JAL', 'KNK', 'LAK', 'MDG', 'NBP', 'SG2', 'RJH', 'SRE','TAM')""", engine)

df.head(10)

,INVOICEDATE,CONTACTPERSONID,FULL_NAME,ACX_MOBILE,ACX_PHONE,INVENTSITEID
0,2024-10-01,P/0026720,JHARNA RANI DUTTA,9433071350,None,SNT
1,2024-10-01,P/0230467,MEENA DEVI,8013383185,None,SL2
2,2024-10-01,P/0324302,TAPAS KUMAR NATH,9483519598,None,GAR
3,2024-10-01,P/0685772,SUMIT GHOSH DASTIDAR,9433465283,9477562915,SOD
4,2024-10-01,P/1591824,NILANJAN SAHA,9433245868,None,GAR
5,2024-10-01,P/2663013,AMIRUL ISLAM CHOWDHURY,9836194807,None,SNT
6,2024-10-01,P/3001330,BASANTA KUMAR SHIT,9874385310,6290282919,GAR
7,2024-10-01,P/3651502,ARUP MANNA,9830812820,None,SNT
8,2024-10-01,P/4559713,JHARNA BISWAS,9681833300,9681833300,SNT
9,2024-10-01,P/5492659,SIBANI DAS,9038715745,9038715745,SOD


#### To create separate Sheets for each INVENTSITEID

In [24]:
# Python allows the use of forward slashes (/) in file paths:

output = 'C:/Users/Shiv n/OneDrive/Desktop/Calling_data.xlsx'   
sheet_col = 'INVENTSITEID'

In [25]:
with pd.ExcelWriter(output, engine= 'openpyxl') as writer:
    for site, group in df.groupby(sheet_col):
        uniq_group = group.drop_duplicates()
        uniq_group = uniq_group.drop_duplicates(subset= ['ACX_MOBILE'])
        uniq_group = uniq_group.sort_values(by= 'INVOICEDATE', ascending= True)
        
        uniq_group.to_excel(writer, sheet_name= str(site), index= False)
        
print(f"Data has been split into seperate sheets in {output}")        

Data has been split into seperate sheets in C:/Users/Shiv n/OneDrive/Desktop/Calling_data.xlsx


#### To create separate Excel workbooks for each INVENTSITEID

In [30]:
output_path = 'C:/Users/Shiv n/OneDrive/Desktop/Site wise patient data/'

In [31]:
import os

os.makedirs(output_path, exist_ok= True)

In [33]:
for site, group in df.groupby(sheet_col):
    uniq_group = group.drop_duplicates()
    uniq_group = uniq_group.drop_duplicates(subset= ['ACX_MOBILE'])
    uniq_group = uniq_group.sort_values(by= 'INVOICEDATE', ascending= True)
        
    file_name = f"{site}.xlsx"
    file_path = os.path.join(output_path, file_name)
    
    uniq_group.to_excel(file_path, engine= 'openpyxl', index= False)
    
print(f"Workbook created for {site} at {output_path}")     

Workbook created for TAM at C:/Users/Shiv n/OneDrive/Desktop/Site wise patient data/
